# Imports

In [1]:
import pandas as pd 
import matplotlib.pyplot as plt
import networkit as nk
import tqdm
import numpy as np
import glob
import sys

from sklearn.metrics import ndcg_score
from sklearn.preprocessing import MinMaxScaler, normalize
from scipy.stats import kendalltau

# Load data

In [2]:
# wikipedia_clickstream_df = pd.concat([pd.read_csv(filepath, sep="\t", on_bad_lines="skip", header=None, names=["prev", "curr", "type", "n"]) for filepath in glob.glob("../data/wikipedia_clickstream/*.tsv")[:2]], ignore_index=True, axis=0)
wikipedia_clickstream_df = pd.read_csv(glob.glob("../data/wikipedia_clickstream/*.tsv")[0], sep="\t", on_bad_lines="skip", header=None, names=["prev", "curr", "type", "n"])
wikipedia_clickstream_df = wikipedia_clickstream_df.loc[wikipedia_clickstream_df["type"]!="external", :]

wikipedia_clickstream_df.head()

,prev,curr,type,n
0,Eddie_Albert,The_Dude_Goes_West,link,17
2,Gale_Storm,The_Dude_Goes_West,link,15
5,Ascoli_Calcio_1898_F.C.,Gianluca_Scamacca,link,87
7,2019–20_Coppa_Italia,Gianluca_Scamacca,link,333
8,2018_UEFA_European_Under-19_Championship,Gianluca_Scamacca,link,23


## Node id and label mapping

In [3]:
%%time

id_label_mapping = {id: label for id, label in enumerate(list(set(wikipedia_clickstream_df["prev"].to_list() + wikipedia_clickstream_df["curr"].to_list())))}
label_id_mapping = {label: id for id, label in id_label_mapping.items()}

CPU times: user 10.8 s, sys: 527 ms, total: 11.4 s
Wall time: 11.4 s


# Generate networkit graph

In [4]:
%%time

kn = 10**5
g = nk.Graph(directed=False)

for row in wikipedia_clickstream_df[["prev", "curr"]].to_records(index=False).tolist()[:kn]:
    g.addEdge(label_id_mapping[row[0]], label_id_mapping[row[1]], addMissing=True)

print("Number of nodes: ", g.numberOfNodes())
print("Number of edges: ", g.numberOfEdges()) 

Number of nodes:  3021398
Number of edges:  100000
CPU times: user 9.75 s, sys: 851 ms, total: 10.6 s
Wall time: 10.6 s


# Network metrics

## Density

In [5]:
print(f"Density: ", nk.graphtools.density(g))

Density:  2.1908581757888712e-08


## Average clustering coefficient

In [6]:
print(f"Average clustering coefficient: ", nk.globals.ClusteringCoefficient().exactGlobal(g))

Average clustering coefficient:  0.0010364814978774662


## Diameter

In [7]:
diameter = nk.distance.Diameter(g, algo=nk.distance.DiameterAlgo.Exact, nSamples=10**5)
diameter.run()

print(f"Diameter: ", diameter.getDiameter())

Diameter:  (22, 0)


# Centrality measures

In [8]:
centrality = {}

## Degree centrality

In [9]:
%%time

centrality["degree"] = nk.centrality.DegreeCentrality(g)
centrality["degree"].run()
[id_label_mapping[item[0]] for item in centrality["degree"].ranking()[:10]]

CPU times: user 1.11 s, sys: 220 ms, total: 1.33 s
Wall time: 751 ms


['Main_Page',
 'The_Guardian',
 'Maldives',
 'Charles_de_Gaulle',
 'Grammy_Award_for_Album_of_the_Year',
 'Kansas',
 'Rwanda',
 'Kidney_failure',
 'Hirohito',
 'Suez_Crisis']

## Closeness centrality

In [10]:
%%time

centrality["closeness"] = nk.centrality.Closeness(g, True, nk.centrality.ClosenessVariant.Generalized)
centrality["closeness"].run()
[id_label_mapping[item[0]] for item in centrality["closeness"].ranking()[:10]]

CPU times: user 13min 3s, sys: 8 s, total: 13min 11s
Wall time: 2min 47s


['Main_Page',
 'Suez_Crisis',
 'The_Guardian',
 'Israeli–Palestinian_conflict',
 'Charles_de_Gaulle',
 'Maldives',
 'Rwanda',
 'Malaria',
 'Reuters',
 'British_Armed_Forces']

## Top-k closeness centrality

In [11]:
%%time

centrality["topkcloseness_0"] = nk.centrality.TopCloseness(g, k=10000, first_heu=False, sec_heu=False)
centrality["topkcloseness_0"].run()
[id_label_mapping[item] for item in centrality["topkcloseness_0"].topkNodesList()[:10]]

CPU times: user 17min, sys: 9.37 s, total: 17min 9s
Wall time: 3min 33s


['Main_Page',
 'Suez_Crisis',
 'The_Guardian',
 'Israeli–Palestinian_conflict',
 'Charles_de_Gaulle',
 'Maldives',
 'Rwanda',
 'Malaria',
 'Reuters',
 'British_Armed_Forces']

In [12]:
%%time

centrality["topkcloseness_1"] = nk.centrality.TopCloseness(g, k=10000, first_heu=False, sec_heu=True)
centrality["topkcloseness_1"].run()
[id_label_mapping[item] for item in centrality["topkcloseness_1"].topkNodesList()[:10]]

CPU times: user 57min 44s, sys: 10min 17s, total: 1h 8min 1s
Wall time: 14min 22s


['Main_Page',
 'Suez_Crisis',
 'The_Guardian',
 'Israeli–Palestinian_conflict',
 'Charles_de_Gaulle',
 'Maldives',
 'Rwanda',
 'Malaria',
 'Reuters',
 'British_Armed_Forces']

# Comparison

## Preprocess

In [13]:
normalised_scores = {}

scaler = MinMaxScaler()
normalised_scores["degree"] = scaler.fit_transform(np.array([row[1] for row in centrality["degree"].ranking()]).reshape(-1, 1)).flatten()

scaler = MinMaxScaler()
normalised_scores["closeness"] = scaler.fit_transform(np.array([row[1] for row in centrality["closeness"].ranking()]).reshape(-1, 1)).flatten()

scaler = MinMaxScaler()
normalised_scores["topkcloseness_0"] = scaler.fit_transform(np.array(centrality["topkcloseness_0"].topkScoresList()).reshape(-1, 1)).flatten()

scaler = MinMaxScaler()
normalised_scores["topkcloseness_1"] = scaler.fit_transform(np.array(centrality["topkcloseness_1"].topkScoresList()).reshape(-1, 1)).flatten()

nodes = {}
nodes["degree"] = [row[0] for row in centrality["degree"].ranking()]
nodes["closeness"] = [row[0] for row in centrality["closeness"].ranking()]
nodes["topkcloseness_0"] = centrality["topkcloseness_0"].topkNodesList()
nodes["topkcloseness_1"] = centrality["topkcloseness_1"].topkNodesList()

## NDCG degree to (top-k) closeness

In [14]:
ndcg_scores = {}

for k in [5, 10, 100, 1000, 10000]:
    ndcg_scores[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        y_score = [normalised_scores[centrality_measure][index] if node in nodes["degree"][:k] else 0 for index, node in enumerate(nodes[centrality_measure][:k])]
        y_true = normalised_scores["degree"][:k]

        ndcg_scores[k][centrality_measure] = ndcg_score([y_score], [y_true])
    
pd.DataFrame(ndcg_scores).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.900184,0.931698,0.931698
10,0.973265,0.980079,0.980079
100,0.967091,0.970014,0.970014
1000,0.987265,0.987528,0.987528
10000,0.958743,0.998283,0.998283


## NDCG (top-k) closeness to degree

In [15]:
ndcg_scores = {}

for k in [5, 10, 100, 1000, 10000]:
    ndcg_scores[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        y_score = [normalised_scores["degree"][index] if node in nodes[centrality_measure][:k] else 0 for index, node in enumerate(nodes["degree"][:k])]
        y_true = normalised_scores[centrality_measure][:k]

        ndcg_scores[k][centrality_measure] = ndcg_score([y_score], [y_true])
    
pd.DataFrame(ndcg_scores).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.985311,0.985311,0.985311
10,0.981775,0.981775,0.981775
100,0.995303,0.995303,0.995303
1000,0.993303,0.993303,0.993303
10000,0.984065,0.984065,0.984065


## Kendall tau degree to (top k) closeness centrality correlation

In [16]:
kendall_tau_corr = {}

for k in [5, 10, 100, 1000, 10000]:
    kendall_tau_corr[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        y_score = [normalised_scores[centrality_measure][index] if node in nodes["degree"][:k] else 0 for index, node in enumerate(nodes[centrality_measure][:k])]
        y_true = normalised_scores["degree"][:k]

        corr, _ = kendalltau([y_true], [y_score])
        kendall_tau_corr[k][centrality_measure] = corr
    
pd.DataFrame(kendall_tau_corr).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.316228,0.316228,0.316228
10,0.787726,0.787726,0.787726
100,0.692692,0.692692,0.692692
1000,0.811134,0.811134,0.811134
10000,0.417640,0.417640,0.417640


## Kendall tau (top k) closeness to degree centrality correlation

In [17]:
kendall_tau_corr = {}

for k in [5, 10, 100, 1000, 10000]:
    kendall_tau_corr[k] = {}
    
    for centrality_measure in ["closeness", "topkcloseness_0", "topkcloseness_1"]:
        y_score = [normalised_scores["degree"][index] if node in nodes[centrality_measure][:k] else 0 for index, node in enumerate(nodes["degree"][:k])]
        y_true = normalised_scores[centrality_measure][:k]

        corr, _ = kendalltau([y_true], [y_score])
        kendall_tau_corr[k][centrality_measure] = corr
    
pd.DataFrame(kendall_tau_corr).T

,closeness,topkcloseness_0,topkcloseness_1
5,0.737865,0.737865,0.737865
10,0.644503,0.644503,0.644503
100,0.820800,0.820800,0.820800
1000,0.703637,0.703637,0.703637
10000,0.218185,0.218185,0.218185
